In [1]:
cd F:\Project\Object Recog. with YOLO

F:\Project\Object Recog. with YOLO


In [2]:
model_path='F:\Project\Object Recog. with YOLO'

In [3]:
min_Confidence=0.4
#Minimum Confidence to filter weak connections
NMS_Threshold=0.3
#Minimum threshold for Non-Max Suppression

In [4]:
#Minimum Distance between 2 persons required
min_Dist=60

In [5]:
import numpy as np
import cv2

In [6]:
def detectPeople(net, frame, ln, personIndx=0):
    results=[]
    (H,W)=frame.shape[:2]
    #Processing frame, and extracting detections using YOLO Algorithm
    blob=cv2.dnn.blobFromImage(frame, 1/255.0, (416,416), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs=net.forward(ln)
    boxes=[]
    confidences=[]
    centers=[]
    for output in layerOutputs:
        for detections in output:
            #Getting score for each probability of detection
            scores=detections[5:]
            #Getting the Prediction with Highest Probability
            classID=np.argmax(scores)
            confidence=scores[classID]
            #Checking if the Highest Probability is that of a person
            if classID==personIndx and confidence>min_Confidence:
                #Getting location of person
                box=detections[0:4]*np.array([W,H,W,H])
                (centerX,centerY,width,height)=box.astype('int')
                top_leftX=int(centerX-(width/2))
                top_leftY=int(centerY-(height/2))
                boxes.append([top_leftX,top_leftY,int(width),int(height)])
                centers.append((centerX,centerY))
                confidences.append(float(confidence))
    #Applying non-max suppression to avoid Extra Boxes
    non_max_suppression=cv2.dnn.NMSBoxes(boxes, confidences, min_Confidence, NMS_Threshold)
    if len(non_max_suppression)>0:
        for i in non_max_suppression.flatten():
            (x1,y1,x2,y2)=(boxes[i][0],boxes[i][1],boxes[i][2]+boxes[i][0],boxes[i][3]+boxes[i][1])
            temp=(confidences[i],(x1,y1,x2,y2),centers[i])
            results.append(temp)
    return results

In [7]:
import imutils
from scipy.spatial import distance as dt
import os

In [8]:
#Required files for YOLO
labelsPath='F:\Project\Object Recog. with YOLO\coco.names'
labels=open(labelsPath).read().strip().split('\n')
weightPaths='F:\Project\Object Recog. with YOLO\yolov3.weights'
configPath='F:\Project\Object Recog. with YOLO\yolov3.cfg'

In [9]:
#Creating YOLO Model from it's files
model=cv2.dnn.readNetFromDarknet(configPath, weightPaths)

In [10]:
cd F:\Project\Social Distancing with YOLO

F:\Project\Social Distancing with YOLO


In [11]:
inp='inp1.mp4'
output='output1.avi'
display=0

In [12]:
ln=model.getLayerNames()
ln=[ln[i[0]-1] for i in model.getUnconnectedOutLayers()]
vs=cv2.VideoCapture(inp)
writer=None

In [13]:
while True:
    (check,frame)=vs.read()
    if not check:
        break
    frame=imutils.resize(frame, width=700)
    results=detectPeople(model, frame, ln, labels.index('person'))
    violations=set()
    #Social Distancing is violated only if Persons>2
    if len(results)>=2:
        centers=np.array([r[2] for r in results])
        D=dt.cdist(centers, centers, metric='euclidean')
        for i in range(0,D.shape[0]):
            for j in range(i+1,D.shape[1]):
                #Checking if Euclidean distance between 2 persons > minimum Distance Required
                if (D[i,j]<min_Dist):
                    violations.add(i)
                    violations.add(j)
    for (i, (conf, box, center)) in enumerate(results):
        (startX, startY, endX, endY)=box
        (centerX,centerY)=center
        color=(0,255,0)
        #Changing Green color to Red as Social Distancing Violated by this person
        if i in violations:
            color=(0,0,255)
        cv2.rectangle(frame, (startX, startY),(endX,endY),color,2)
    #Saving Processed Video as Output
    if writer==None and output!='':
        fourcc=cv2.VideoWriter_fourcc(*'MJPG')
        writer=cv2.VideoWriter(output, fourcc, 25, (frame.shape[1],frame.shape[0]),True)
    if writer is not None:
        writer.write(frame)
writer.release()